<a href="https://colab.research.google.com/github/Nilanshrajput/AdvitiyaHackathon/blob/master/Evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Nilanshrajput/AdvitiyaHackathon.git

Cloning into 'AdvitiyaHackathon'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 312 (delta 15), reused 15 (delta 6), pack-reused 282
Receiving objects: 100% (312/312), 1.68 MiB | 271.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


In [3]:
!pip install transformers
!pip install beautifulsoup4==4.8.2

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import numpy as np
import pandas as pd
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn

from time import time

import pandas as pd
import re 
import matplotlib.pyplot as plt
# ~ Import packages ~ #
from AdvitiyaHackathon.google_patent_scrapper import scraper_class
import json
import os
import spacy

from nltk.corpus import stopwords
import string
import spacy #load spacy

import nltk
nltk.download('stopwords')

from transformers import BertForSequenceClassification, AdamW, BertConfig


In [0]:
drive_pt="/content/drive/My Drive/dl_projects/AdvitiyaHackathon/"

git_path = "AdvitiyaHackathon/"
path = drive_pt+"ScrappeData"

In [0]:
%%capture
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm



In [0]:
#class for scraping and cleaning data
class data_wrangler:
  def __init__(self, data_csv_pt=None,patent = None, eval = False, save = True, max_limit = 10000, classification = True, abstract =False, claims  = False, all = False,
               path = "/content/drive/My Drive/dl_projects/AdvitiyaHackathon/ScrappeData",
               test_path = "/content/drive/My Drive/dl_projects/AdvitiyaHackathon/Test_scrap",
               val = False, test = False):
    self.patent_info = ""
    self.path = path
    self.patent =patent
    self.eval = eval
    self.save = save
    self.test_path = test_path
    self.max_limit = max_limit
    self.class_names = ["non_alc", "alc", "non_auto", "auto"]
    self.info_classes = ["title", 'classification' ]
    self.data_csv_pt = data_csv_pt
    if self.data_csv_pt:
      self.csv  = pd.read_csv(self.data_csv_pt,index_col= 0)
    
    self.scraper = scraper_class()
    

    self.nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
    self.nlp.max_length = 50000000000
    self.stops = stopwords.words("english")
    self.punctuations = string.punctuation

    self.cleaned_data  = pd.DataFrame(columns= ["text","cat"])
    self.cleaned_data_test  = pd.DataFrame(columns= ["text","cat"])

    if abstract :
      self.info_classes = ["title", "abstract", "claims", 'classification' ]
    elif claims :
      self.info_classes = ["title", "claims", 'classification', "abstract" ]
    elif classification :
      self.info_classes = ["title", 'classification', "abstract", "claims" ]

    else :
      self.info_classes = ["title", 'classification', "abstract", "claims" ]
    
    if eval :
      self.scrappedJson = self.scraper.scrap_single_patent(patent)
      try:  
        self.patent_info = ""
        for info in self.info_classes:
          self.patent_info += self.scrappedJson[info]
          if len(self.patent_info.split()) > self.max_limit:
            break
      except Exception as e:
        print(e)
      self.patent_info = self.clean(self.patent_info)
      self.patent_info = re.sub(' +', ' ',str(self.patent_info))
    else:

      if test:
        self.test_csv = pd.read_csv(self.data_csv_pt)
        self.test_nums = list(self.test_csv.iloc[:,0])
        self.test_json_pt = [os.path.join(self.test_path,p_num+".json") for p_num in self.test_nums]
        try: 
        
          for patent in tqdm(self.test_json_pt,desc  = "patents"):
              with open(patent) as file:
                  data = json.load(file)
              try:  
                self.patent_info = ""
                for info in self.info_classes:
                  self.patent_info += data[info]
                  if len(self.patent_info.split()) > self.max_limit:
                    break
              except Exception as e:
                print(e)
              self.patent_info = self.clean(self.patent_info)
              self.patent_info = re.sub(' +', ' ',str(self.patent_info))
              self.cleaned_data_test = self.cleaned_data_test.append({"text":str(self.patent_info),"cat":0},ignore_index = True)
        

        except Exception as e:
          print(e)

        self.cleaned_data_test.to_csv("final_test_clean.csv")
      

        
      else:

        # Make a separate list for each category
        self.non_alc = list(self.csv.iloc[:,0])
        self.alc = list(self.csv.iloc[:,2])
        self.non_auto = list(self.csv.iloc[:,4])
        self.auto = list(self.csv.iloc[:,6])

        self.catwise_patent_lists  = [self.non_alc, self.alc, self.non_auto, self.auto]

        # Make a separate list for each category
        self.non_alc_json_pt = [os.path.join(self.path,self.class_names[0],p_num+".json") for p_num in self.catwise_patent_lists[0]]
        self.alc_json_pt = [os.path.join(self.path,self.class_names[1],p_num+".json") for p_num in self.catwise_patent_lists[1]]
        self.non_auto_json_pt = [os.path.join(self.path,self.class_names[2],p_num+".json") for p_num in self.catwise_patent_lists[2]]
        self.auto_json_pt = [os.path.join(self.path,self.class_names[3],p_num+".json") for p_num in self.catwise_patent_lists[3]]

      
        i = 0
        for pt_list in tqdm([self.non_alc_json_pt,self.alc_json_pt,self.non_auto_json_pt, self.auto_json_pt], desc = "CatList"):
        
          try: 
          
            for patent in tqdm(pt_list,desc  = "patents"):
                with open(patent) as file:
                    data = json.load(file)
                try:  
                  self.patent_info = ""
                  for info in self.info_classes:
                    self.patent_info += data[info]
                    if len(self.patent_info.split()) > self.max_limit:
                      break
                except Exception as e:
                  print(e)
                self.patent_info = self.clean(self.patent_info)
                self.patent_info = re.sub(' +', ' ',str(self.patent_info))
                self.cleaned_data = self.cleaned_data.append({"text":str(self.patent_info),"cat":i},ignore_index = True)
            i += 1

          except Exception as e:
            print(e)
      if self.save: 
          if val:
            self.cleaned_data.to_csv("final_val_clean.csv")
          else:
            self.cleaned_data.to_csv("final_train_clean.csv")

  def get_data(self):
    if self.eval:
      return self.patent_info
    else:
      return self.cleaned_data

  def clean(self, comment, lowercase = True, remove_stopwords = True):
    if lowercase:
        comment = comment.lower()
    comment = self.nlp(comment,disable = ['ner', 'parser', 'tagger'])
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
       
        #len(re.findall('[^A-Za-z0-9]+',lemma))==0 for removing alphanumerals in b/w strings

        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in self.stops and not ("\\" in r"%r" % lemma)  ):
                lemma = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(lemma))
                lemmatized.append(lemma)
    return " ".join(lemmatized)


In [0]:
#class for predictin and evaluating models


class Evaluate:
    def __init__(self,model_paths):
        
        self.model_paths = model_paths
        self.model = BertForSequenceClassification.from_pretrained(
                        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
                        num_labels = 4, # The number of output labels--2 for binary classification.
                                        # You can increase this for multi-class tasks.   
                        output_attentions = True, # Whether the model returns attentions weights.
                        output_hidden_states = False, # Whether the model returns all hidden-states.
                    )
        self.model.cuda()
        
        # If there's a GPU available...
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")
        
        self.models = [self.model for i in range(len(self.model_paths))]
        i=0
        for path in tqdm(self.model_paths, desc = "Loading Models"):
            # Loading the checkpoints for resuming training
            checkpoint = torch.load(path)
            self.models[i].load_state_dict(checkpoint['model_state_dict'])
  
            self.models[i].eval()
            i+=1
            
            
    def predict(self, patent):
        self.patent_list = patent
        self.extractor  = data_wrangler(eval = True,patent = patent,save=False)
        sentence = self.extractor.get_data()
        b_input_ids, b_input_mask = self.pre_process(sentence=sentence, maxlen = 512,device=self.device)
        results = []
        for model in tqdm(self.models):
            output = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)
            logits = output[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            cat = np.argmax(logits, axis=1).flatten()
            results.append(int(cat))
        cat  = self.most_frequent(results)
        return cat

    def most_frequent(self, List): 
        counter = 0
        num = List[0] 

        for i in List: 
            curr_frequency = List.count(i) 
            if(curr_frequency> counter): 
                counter = curr_frequency 
                num = i 

        return num 



    def pre_process(self, sentence, maxlen,device):
      tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      #sentence = sentence[50:]
      tokens_ids = tokenizer.encode(str(sentence),add_special_tokens = True,max_length = maxlen, pad_to_max_length=True)
      # Converting the list to a pytorch tensor
      tokens_ids_tensor = torch.tensor(tokens_ids)

      # Obtaining the attention mask
      attn_mask = (tokens_ids_tensor != 0).long()

      return tokens_ids_tensor.unsqueeze(dim = 0).to(device), attn_mask.unsqueeze(dim = 0).to(device)
        


In [0]:
save_path = drive_pt+ "Models/"

paths = [save_path+'all_final_calims_300_16_20.pth',save_path+'all_final_calims_128_32_30.pth',save_path+'finalbertv4_claims_512_8_20.pth',save_path+'all_final_200_32_30.pth']

In [18]:
predictor = Evaluate(paths)

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [21]:
out = predictor.predict("US20150086688A1")
print(out)

0
